# Basic RAG from existing index

## Setup

In [46]:
import os

from openai import AzureOpenAI

import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from chromadb.config import Settings

from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

from IPython.display import Markdown, display

from openai import OpenAI, AzureOpenAI

from dotenv import load_dotenv
load_dotenv()

True

In [33]:
openai_client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002",
    api_type="azure",
    api_version="2024-05-01-preview"
)

chroma_client_load = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb",
    settings=Settings(allow_reset=True)
)


## Load collection

In [34]:
collection_load = chroma_client_load.get_collection(name="landsforsoeg", embedding_function=openai_ef)

## Prompt

In [50]:
prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the content of documents and provides detailed expert advice. 
    You must provide your answer in the Danish language.
    If the answer contains multiple steps or points, provide the answer in a bullet format.
    Below the answer, the source of the answer should be provided including file_name and page number.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query}
    Answer: 
    """,
)

## Queries

In [56]:
query = "hvem udgiver landsforsøgene?"
#query = "hvem udgiver landsforsøgene?"
#query = "hvordan kan jeg bedst bekæmpe væselhale?"
query = "hvordan vælger jeg den bedste vårbygsort?"
#query = "hvad er reglerne for afstande ved etablering af husdyranlæg?"
query = "Beskriv MT-Klovvask"

In [57]:
result = collection_load.query(query_texts=[query], n_results=5)
context = result["documents"][0]

message = prompt.format(query=query, context="\n\n".join(context))

In [58]:
stream = openai_client.chat.completions.create(
    #messages=[{"role": "user", "content": query}],
    model="gpt4",
    messages=[
        {"role": m["role"], "content": m["content"]}
        for m in [{"role": "user", "content": message}]#st.session_state.messages
    ],
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

- MT-Klovvask består af en bund udformet af dørkplader i aluminium dækket med gummigulv og siderne er brune krydsfinerplader. 
- Sider er indsat for at forhindre køerne i at sidde fast eller skade benene ved at træde ud over siden. 
- Klovene spules bagfra med fire kraftige stråler af vand, som kan kombineres med spuling fra to dyser forfra.
- MT-Klovvasken kan placeres før eller efter malkerområdet.
- Det fungerer ved hjælp af koldt vand under tryk og spuler klovene med 5-7 liter vand i sekundet. 
- Vandet recirkulerer fra en tank under klovvaskeren og kan automatisk skiftes efter individuel tidsindstilling, og tanken kan spules ren. 
- Der er en sensor, der aktiverer klovvaskeren, lige før en ko går igennem, og den stopper 1,5 minutter efter sidste ko.

Kilde: FarmTest Kvæg nr. 73, side 12.

## Query engine

In [36]:
class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: AzureOpenAI
    prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content(metadata_mode="all") for n in nodes])
        #context = qa_prompt.format(
        #    context_str=context_str, query_str=query_str)
        response = self.llm.complete(
            qprompt.format(context_str=context_str, query_str=query_str)
        )
                    
        return str(response) + "\n\n-------------------------\n\nKontekst:\n\n" + context_str


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    prompt=prompt,
)

NameError: name 'index' is not defined